In [57]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import re

In [58]:
# https://github.com/mrodr273/Veggies_GrowingGuide/blob/93998d60434a0166d5a1ff4276f9b34594b5fa95/BeautifulSoup.ipynb
# https://github.com/mrodr273/Veggies_GrowingGuide/blob/93998d60434a0166d5a1ff4276f9b34594b5fa95/Veggies_GrowingGuide_WebScapping.ipynb

In [59]:
with urllib.request.urlopen('http://www.gardening.cornell.edu/homegardening/scene0391.html') as response:
                            html = response.read()

In [60]:
soup = BeautifulSoup(html,'lxml')
pages = []
for link in soup.find_all('a')[8:-1]:
    pages.append(str(link))

In [61]:
pages_formated = []
for page in pages:
    start_scene = page.find("href") + len("href")+2
    end_scene = page.find(".html")
    substring_scene = page[start_scene:end_scene]

    start_veggie = page.find(".html")+len(".html")+2
    end_veggie = page.find('</a>')
    substring_veggie = page[start_veggie:end_veggie]

    page_formated = "<a href=\""+substring_scene+".html\">"+substring_veggie+'</a>'
    pages_formated.append(page_formated)

In [62]:
scenes = []
veggies = []
for page in pages_formated:
    
    pattern =  "<a href=\"(.*?)\">(.*?)</a>"

    scene = re.search(pattern, page).group(1)
    vegetable = re.search(pattern, page).group(2)
    scenes.append(scene)
    veggies.append(vegetable)

In [63]:
url_base = "http://www.gardening.cornell.edu/homegardening/"
url_scenes = []
for scene in scenes:
    url_scene = url_base+scene
    url_scenes.append(url_scene)

In [64]:
soups = []
for url in url_scenes:
    with urllib.request.urlopen(url) as response:
                            html = response.read()
    soup = BeautifulSoup(html,'lxml')
    soups.append(soup)

In [65]:
veg_seasons = []
veg_scientific_names = []
veg_family = []
veg_description = []
for soup in soups:
    basic_info = soup.find("div", {"class": "normal"})
    season = basic_info.find('p').get_text().strip()
    scientific_name = basic_info.find('i').get_text().strip()
    pattern =  "(.*?) Family"
    family = re.search(pattern, basic_info.get_text()).group(1)
    description = basic_info.find_all('p')[2].get_text().strip()
    veg_seasons.append(season)
    veg_scientific_names.append(scientific_name)
    veg_family.append(family)
    veg_description.append(description)

In [66]:

sunlight_all = []
soil_condition_all = []
hardiness_all = []
lifecycle_all = []
ease_care = []
foliage_color_all = []
foliage_texture_all = []
tolerates_all = []
special_charac_all = []
for soup in soups:
    if soup.find(text="Sunlight:") != None:
        sunlight = soup.find(text="Sunlight:").next_element.next_element.get_text().strip()
        sunlight_all.append(sunlight)
    else:
        sunlight_all.append(None)
    
    if soup.find(text="Soil conditions:") != None:
        soil_condition = soup.find(text="Soil conditions:").next_element.next_element.next_element.get_text().strip()
        soil_condition_all.append(soil_condition)
    else:
        soil_condition_all.append(None)
    
    if soup.find(text="Hardiness zones:")!= None:
        hardiness = soup.find(text="Hardiness zones:").next_element.next_element.next_element.get_text().strip()
        hardiness_all.append(hardiness)
    else:
        hardiness_all.append(None)
    
    if soup.find(text="Lifecycle:") != None:
        lifecycle = soup.find(text="Lifecycle:").next_element.strip()
        lifecycle_all.append(lifecycle)
    else:
        lifecycle_all.append(None)
    
    if soup.find(text="Ease-of-care:") != None:
        ease_of_care = soup.find(text="Ease-of-care:").next_element.strip()
        ease_care.append(ease_of_care)
    else:
        ease_care.append(None)

    if soup.find(text="Foliage color:") != None:
        foliage_color = soup.find(text="Foliage color:").next_element.strip()
        foliage_color_all.append(foliage_color)
    else:
        foliage_color_all.append(None)

    if soup.find(text="Foliage texture:") != None:
        foliage_texture = soup.find(text="Foliage texture:").next_element.strip()
        foliage_texture_all.append(foliage_texture)
    else:
        foliage_texture_all.append(None)

    if soup.find(text="Tolerates:") != None:
        tolerates = soup.find(text="Tolerates:").next_element.next_element.get_text().strip()
        tolerates_all.append(tolerates)
    else:
        tolerates_all.append(None)
        
    if soup.find(text="Special characteristics:") != None:
        special_charac = soup.find(text="Special characteristics:").next_element.next_element.get_text().strip()
        special_charac_all.append(special_charac.replace('\r','').replace('\n','').replace('\t',''))
    else:
        special_charac_all.append(None)

In [67]:
how_to_grow = []
germination_temp = []
emergence_days = []
seed_save = []

for soup in soups:
    item = soup.find(text="How to plant:").next_element.next_element.get_text().strip().replace('\t','').replace('\r','').split('\n')
    while("" in item) : 
        item.remove("")
    listToStr = ' '.join([str(element) for element in item]) 
    listToStr = listToStr.replace('Germination temperature', '').replace('Days to emergence','')
    grow_info=re.split('[:]',listToStr)
    how_to_grow.append(grow_info[0].replace('  ',''))
    #fix comparison
    if (len(grow_info) > 1):
        germination_temp.append(grow_info[1])
    else:
        germination_temp.append('')
    if (len(grow_info)>2):
        piece = grow_info[2]
        piece_start = piece.find('Seed can be saved')
        if (piece_start)!= -1:
            emergence_days.append(piece[:piece_start])
            seed_save.append(piece[piece_start:])
        else:
            emergence_days.append(piece)
            seed_save.append('')
    else:
        emergence_days.append('')
        seed_save.append('')

In [68]:
pests_info = []
maintenance_info = []
for soup in soups:
    if soup.find(text="Maintenance and care:") != None:
        maintenance = soup.find(text="Maintenance and care:").next_element.next_element.get_text().strip().replace('\r','').replace('\n','')
        maintenance_info.append(maintenance)
    else:
        maintenance_info.append('')
    if soup.find(text="Pests:") != None:
        pests = soup.find(text="Pests:").next_element.next_element.get_text().replace('\r','').replace('\n','').replace('\t','')
        pests_info.append(pests)
    else:
        pests_info.append('')

In [69]:
d = {'veggie': veggies,'scene':scenes,'url_scene':url_scenes,'season':veg_seasons,'scientific_name':veg_scientific_names,'family':veg_family,'description': veg_description, 'sunlight':sunlight_all,'soil conditions':soil_condition_all,'hardiness':hardiness_all,'lifecycle':lifecycle_all,'ease of care':ease_care,'foliage colors':foliage_color_all,'foliage texture':foliage_texture_all,'tolerates':tolerates_all,'special characteristics':special_charac_all,'how to grow':how_to_grow,'germination temperature':germination_temp,'emergence days':emergence_days,'seed save time':seed_save,'Maintenance Info':maintenance_info,'Pests':pests_info}
df = pd.DataFrame(data=d)

In [70]:
df.to_csv('veggies_df.csv')

In [71]:
url = 'https://www.almanac.com/gardening/planting-calendar/zipcode/92861'
req = urllib.request.Request(url,headers={'User-Agent' : 'Mozilla/5.0'})
webpage = urllib.request.urlopen(req).read()

In [72]:
page = BeautifulSoup(webpage,'html.parser')

In [73]:
pieces = page.find_all("tr", {"class": "plantrow"})
pieces_plants = []
for piece in pieces:
    pieces_plants.append(piece.find('a').get('href'))

In [74]:
veggies_almanac = []
for piece in pieces_plants:

    index_end = piece.rfind('/')
    veggies_almanac.append(piece[index_end+1:].capitalize())

In [75]:
fall_veggies = veggies_almanac[:27]
spring_veggies = veggies_almanac[27:]
calendar_info = page.find_all("td")
dates = []

for info in calendar_info:
    dates.append(info.get_text())

In [76]:
fall_dates = dates[:81]
seed_indoors_fall = [fall_dates[x] for x in range(0, len(fall_dates), 3)]
plant_outdoors_fall= [fall_dates[x+1] for x in range(0, len(fall_dates), 3)]
start_outdoors_fall = [fall_dates[x+2] for x in range(0, len(fall_dates), 3)]

In [77]:
d_fall = {'veggie': fall_veggies ,'seed indoors':seed_indoors_fall,'plant outdoors':plant_outdoors_fall,'start indoors': start_outdoors_fall}
df_fall_calendar = pd.DataFrame(data=d_fall)

In [78]:
spring_dates = dates[81:]
seed_indoors_spring = [spring_dates[x] for x in range(0, len(spring_dates), 3)]
plant_seedlings_spring = [spring_dates[x+1] for x in range(0, len(spring_dates), 3)]
start_outdoors_spring = [spring_dates[x+2] for x in range(0, len(spring_dates), 3)]

In [79]:
d_spring = {'veggie': spring_veggies ,'seed indoors':seed_indoors_spring,'plant seedlings':plant_seedlings_spring,'start indoors': start_outdoors_spring}
df_spring_calendar = pd.DataFrame(data=d_spring)

In [80]:
df_fall_calendar.to_csv('fall_calendar_df.csv')
df_spring_calendar.to_csv('spring_calendar_df.csv')